# Notebook für SDM Analyse

## Load packages

In [23]:
# import standard packages
from datetime import date, datetime
import math
import os
from pathlib import Path

import ee
import fiona
import geemap #.foliumap as geemap
import geemap.foliumap as folium
import geemap.colormaps as cm
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np


from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import pandas as pd
from pygbif import species as species
from pygbif import occurrences as occ

import rioxarray as rxr
import xarray as xr
import leafmap



from toolbox.utils import *

## GEE Auth

In [2]:
ee.Authenticate()
# enter your project id here
# check google on how to set up a project
ee.Initialize(project='ee-sebasd1991')

## Parameter setup

In [56]:
year = 2024

country_code = 'AT'
county_name = 'Tirol'

species_name = 'Lagopus muta'

selected_features = ['landcover', 'CHM', 'b1', 'b12']

### Model setup

In [57]:
model = 'Random Forest' # Maxent, Embedding
n_trees  = 300
tree_depth = 5
train_size = .85

In [30]:
country, county = get_aoi_from_nuts(country_code='AT', county_name='Tirol')
species_df = get_species_data(species_name, country_code)
species_ee = geemap.gdf_to_ee(species_df)

INFO:Created 1 records
INFO:Created 1 records
INFO:Created 300 records


## Load all prediction layer

In [31]:
# WorldClim Bioclim
try:
    bioclim = ee.Image("projects/ee-sebasd1991/assets/BioClim")
except:
    print("Bioclim asset not found, loading history data from pubic source.")
    bioclim = ee.Image("WORLDCLIM/V1/BIO")
 
# NASA SRTM Digital Elevation 30m
terrain = ee.Algorithms.Terrain(ee.Image("USGS/SRTMGL1_003"))
northness = terrain.select('aspect').multiply(math.pi/180).cos().rename('northness')
eastness = terrain.select('aspect').multiply(math.pi/180).sin().rename('eastness')

# NDVI
NDVI = (
    ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI')
    .filterDate(f'{year}-01-01', f'{year + 1}-01-01')
    .mean()
).rename('NDVI')

HM_300M = ee.ImageCollection("projects/sat-io/open-datasets/GHM/HM_2022_300M").first().rename('GHMI')

era5 = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
        .filter(ee.Filter.date(f'{year}-01-01', f'{year + 1}-01-01'))
)
swe = era5.select(['snow_depth_water_equivalent'], ['SWE']).mean()
snow_depth = era5.select(['snow_depth'], ['snow_depth']).mean()
snow_cover = era5.select(['snow_cover'], ['snow_cover']).mean()
snow_albedo = era5.select(['snow_albedo'], ['snow_albedo']).mean()

#CLC
landcover = ee.Image("COPERNICUS/CORINE/V20/100m/2018").select('landcover')
canopyHeight = ee.ImageCollection("projects/sat-io/open-datasets/facebook/meta-canopy-height").mosaic().rename('CHM');

nari = gee_calculate_scrub_index('nari', year).rename('NARI')
ncri = gee_calculate_scrub_index('ncri', year).rename('NCRI')

In [32]:
# Combine bands into a multi-band image
predictors = landcover.addBands(northness).addBands(eastness).addBands(NDVI).addBands(swe).addBands(snow_cover).addBands(snow_depth).addBands(terrain.select(['elevation', 'aspect', 'slope'])).addBands(canopyHeight).addBands(nari).addBands(ncri).addBands(HM_300M).addBands(bioclim)

# Create a water mask
watermask = terrain.select('elevation').gt(0)

# Mask out ocean pixels and clip to the area of interest
predictors = predictors.updateMask(watermask).clip(country)

In [44]:
background_data = load_background_data()
presence_data, predictors = get_species_features(species_df, selected_features, get_layer_information(year))

INFO:Created 300 records


In [58]:
model, results_df, ml_gdf = compute_sdm(
                                presence=presence_data,
                                background=background_data,
                                features=list(selected_features),
                                model_type=model, 
                                n_trees=n_trees, 
                                tree_depth=tree_depth, 
                                train_size=train_size, 
                                )

In [59]:
classified_img_pr = classify_image_aoi(
                            image=predictors,
                            aoi=county,
                            ml_gdf=ml_gdf,
                            model=model,
                            features=list(selected_features)
                        )

INFO:Created 490 records


In [61]:
results_df

,roc_auc,landcover,CHM,b1,b12
0,0.963816,0.031552,0.099885,0.248842,0.619721
1,0.929104,0.053265,0.085472,0.264135,0.597127
2,0.972143,0.026700,0.079517,0.336955,0.556828
3,0.947667,0.037765,0.078170,0.306270,0.577795
4,0.959629,0.012936,0.165910,0.231890,0.589264
5,0.946793,0.023546,0.080777,0.305401,0.590276
6,0.973431,0.034006,0.107146,0.322228,0.536620
7,0.961861,0.026854,0.131358,0.258235,0.583553
8,0.972534,0.015819,0.164529,0.269616,0.550037
9,0.939225,0.027578,0.110267,0.340980,0.521175


## Show map

In [62]:
Map = geemap.Map()
Map.add_basemap("SATELLITE")

Map.addLayer(classified_img_pr, {'min': 0, 'max': 1, 'palette': geemap.colormaps.palettes.viridis_r}, 'Classified Image')

Map.add_colorbar({'min': 0, 'max': 1, 'palette': geemap.colormaps.palettes.viridis_r}, label="Habitat suitability",
                orientation="vertical",
                position="bottomright",
                layer_name="Habitat suitability")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…